In [ ]:
!pip install mido
!pip install tensorflow-addons
!pip install --upgrade --ignore-installed tensorflow
!pip install -q imageio
!pip install -q git+https://github.com/tensorflow/docs

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.1/51.1 kB 994.5 kB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 591.0/591.0 kB 9.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 585.9/585.9 MB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.9/22.9 MB 73.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 109.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 70.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.5/126.5 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 

In [ ]:
# !pip uninstall -y numpy
!pip install numpy==1.23.* --ignore-installed

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached numpy-1.23.5-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.1 MB)


In [ ]:
# import mido
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

import librosa

import pandas as pd
import os

import tensorflow_addons as tfa
from tensorflow.keras import layers

from IPython import display
from IPython.display import clear_output

import glob
import imageio
import time
import IPython.display as ipd

AUTOTUNE = tf.data.experimental.AUTOTUNE

/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [ ]:
seed=123
tf.compat.v1.set_random_seed(seed)
session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
tf.compat.v1.keras.backend.set_session(sess)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train_size = 60000
BATCH_SIZE = 10
test_size = 10000
epochs = 20
# set the dimensionality of the latent space to a plane for visualization later
latent_dim = 2
num_examples_to_generate = 10

BASE_PATH = '/content/drive/My Drive/genres_original'

In [ ]:
def DatasetLoader(class_):
    music_list = np.array(sorted(os.listdir(BASE_PATH+'/'+class_)))
    train_music_1 = list(music_list[[0,52,19,39,71,12,75,85,3,45,24,46,88]]) #99,10,66,76,41
    train_music_2 = list(music_list[[4,43,56,55,45,31,11,13,70,37,21,78]]) #65,32,53,22,19,80,89,
    TrackSet_1 = [(BASE_PATH)+'/'+class_+'/%s'%(x) for x in train_music_1]
    TrackSet_2 = [(BASE_PATH)+'/'+class_+'/%s'%(x) for x in train_music_2]

    return TrackSet_1, TrackSet_2

In [ ]:
def load(file_):
    data, sampling_rate = librosa.load(file_,sr=3000, offset=0.0, duration=30)
    data = data[0 : 75000]
    data = data.reshape(100, 750)
    data = tf.convert_to_tensor(data, dtype = tf.float32)
    # data_ = data_[0 : 90000]
    # data = []
    # for i in range(120):
    #   arg = tf.convert_to_tensor(data_[750*i:750*(i+1)], dtype = tf.float32)
    #   arg = tf.reshape(arg, shape=(1, 750))
    #   data.append(arg)
    return data
map_data = lambda filename: tf.compat.v1.py_func(load, [filename], tf.float32)

In [ ]:
TrackSet_1, TrackSet_2 = DatasetLoader('jazz')

In [ ]:
samp = load(TrackSet_1[1])
samp

<tf.Tensor: shape=(100, 750), dtype=float32, numpy=
array([[ 0.01675216,  0.07342426, -0.12623996, ..., -0.13352165,
        -0.11601178, -0.01350737],
       [-0.05094993, -0.11805218, -0.06304592, ..., -0.1811176 ,
        -0.10112785,  0.01415266],
       [-0.00813589,  0.02537277,  0.07389332, ..., -0.22472794,
        -0.23269588, -0.28209594],
       ...,
       [ 0.031294  , -0.0033436 , -0.0111826 , ...,  0.07747798,
        -0.00338306, -0.04828962],
       [-0.01585888, -0.01529859,  0.04896452, ...,  0.21051843,
         0.323723  ,  0.25154477],
       [ 0.16953471,  0.08796553,  0.06990232, ...,  0.1895045 ,
        -0.09651382, -0.15970421]], dtype=float32)>

In [ ]:
class Resnet1DBlock(tf.keras.Model):

    def __init__(self, kernel_size, filters, type='encode'):
        super(Resnet1DBlock, self).__init__()
        if type=='encode':
            self.conv1a = layers.Conv1D(filters, kernel_size, 2,padding="same")
            self.conv1b = layers.Conv1D(filters, kernel_size, 1,padding="same")
            self.norm1a = tfa.layers.InstanceNormalization()
            self.norm1b = tfa.layers.InstanceNormalization()
        if type=='decode':
            self.conv1a = layers.Conv1DTranspose(filters, kernel_size, 1,padding="same")
            self.conv1b = layers.Conv1DTranspose(filters, kernel_size, 1,padding="same")
            self.norm1a = tf.keras.layers.BatchNormalization()
            self.norm1b = tf.keras.layers.BatchNormalization()
        else:
            return None

    def call(self, input_tensor):
        x = tf.nn.relu(input_tensor)
        x = self.conv1a(x)
        x = self.norm1a(x)
        x = layers.LeakyReLU(0.4)(x)
        x = self.conv1b(x)
        x = self.norm1b(x)
        x = layers.LeakyReLU(0.4)(x)
        x += input_tensor
        return tf.nn.relu(x)

In [ ]:
class VAE(tf.keras.Model):

    def __init__(self, latent_dim):
        super(VAE, self).__init__()
        self.latent_dim = latent_dim
        self.encoder = tf.keras.Sequential(
            [
                # layers.Reshape((1, 1), input_shape = (750, )),
                tf.keras.layers.InputLayer(input_shape=(1,750)),
                layers.Conv1D(64,1,2),
                Resnet1DBlock(64,1),
                layers.Conv1D(128,1,2),
                Resnet1DBlock(128,1),
                layers.Conv1D(128,1,2),
                Resnet1DBlock(128,1),
                layers.Conv1D(256,1,2),
                Resnet1DBlock(256,1),
                layers.Flatten(),
                layers.Dense(2 * latent_dim)
            ]
        )
        self.decoder = tf.keras.Sequential(
            [
                layers.InputLayer(input_shape=(latent_dim, )),
                layers.Reshape(target_shape=(1, latent_dim)),
                Resnet1DBlock(512, 1, 'decode'),
                layers.Conv1DTranspose(512, 1, 1),
                Resnet1DBlock(256, 1, 'decode'),
                layers.Conv1DTranspose(256, 1, 1),
                Resnet1DBlock(128, 1, 'decode'),
                layers.Conv1DTranspose(128, 1, 1),
                Resnet1DBlock(64, 1, 'decode'),
                layers.Conv1DTranspose(64, 1, 1),
                layers.Conv1DTranspose(750, 1, 1)
            ]
        )
        self.next_state = tf.keras.Sequential(
            [
                tf.keras.layers.InputLayer(input_shape = (latent_dim, )),
                layers.Reshape(target_shape=(1, latent_dim)),
                layers.Dense(256),
                layers.LeakyReLU(0.4),
                layers.Dense(128),
                layers.LeakyReLU(0.4),
                layers.Flatten(),
                layers.Dense(2 * latent_dim)
            ]
        )
    @tf.function
    def sample(self, eps=None):
        if eps is None:
            eps = tf.random.normal(shape=(200, self.latent_dim))
        return self.decode(eps, apply_sigmoid=True)
    @tf.function
    def encode(self, x):
        mean, logvar = tf.split(self.encoder(x), num_or_size_splits=2, axis=1)
        return mean, logvar
    @tf.function
    def reparametrize(self, mean, logvar):
        eps = tf.random.normal(shape=mean.shape)
        return eps * tf.exp(logvar * .5) + mean
    @tf.function
    def decode(self, z, apply_sigmoid=False):
        logits = self.decoder(z)
        if apply_sigmoid:
            probs = tf.sigmoid(logits)
            return probs
        return logits
    @tf.function
    def get_next_state(self, z_prev):
        mean, logvar = tf.split(self.next_state(z_prev), num_or_size_splits=2, axis=1)
        return mean, logvar

In [ ]:
test_samp = tf.reshape(samp[0], shape = (1, 1, 750))
z_test = model.encode(test_samp)

In [ ]:
z_test

(<tf.Tensor: shape=(1, 300), dtype=float32, numpy=
 array([[ 0.04627123,  0.03461462,  0.02363694,  0.00360415,  0.0405947 ,
         -0.10060225, -0.02886691,  0.01631321, -0.01584181, -0.00685004,
         -0.067425  ,  0.01542851, -0.04465303,  0.0019353 ,  0.01011637,
          0.00204912,  0.03390533,  0.03462849,  0.01733246, -0.12308466,
         -0.05646379, -0.02011564,  0.00971696,  0.02974458,  0.03742634,
          0.00715089, -0.04241169,  0.01552719, -0.00357116,  0.02194808,
         -0.01403634, -0.00543231, -0.03094707, -0.04317864, -0.07675375,
         -0.02384648,  0.01906503,  0.00313196,  0.02423473,  0.03885249,
         -0.02099917,  0.04133912,  0.01408148,  0.00060269, -0.01381935,
          0.00897847,  0.02751249, -0.0334162 ,  0.0840389 ,  0.02581413,
         -0.03058638,  0.05501803, -0.01120817, -0.04150539,  0.01315274,
          0.02347379, -0.04402716,  0.01758355,  0.00855046, -0.03256001,
          0.01018918, -0.00825097, -0.03317941, -0.03119295, 

In [49]:
optimizer = tf.keras.optimizers.legacy.Adam(0.00001, beta_1=0.9, beta_2=0.999,epsilon=1e-07)

In [ ]:
@tf.function
def log_normal_pdf(sample, mean, logvar, raxis=1):
    log2pi = tf.math.log(2. * np.pi)
    return tf.reduce_sum(
         -.5 * ((sample - mean) ** 2. * tf.exp(-logvar) + logvar + log2pi),
          axis=raxis)

In [ ]:
@tf.function
def compute_loss(model, test_sample):
    total_loss = 0
    for y in test_sample:
        x = tf.reshape(y, shape = (1, 1, 750))
        mean, logvar = model.encode(x)
        z = model.reparametrize(mean, logvar)
        x_logit = model.decode(z)
        cross_ent = tf.nn.sigmoid_cross_entropy_with_logits(logits=x_logit, labels=x)
        logpx_z = -tf.reduce_sum(cross_ent, axis=[1,2])
        logpz = log_normal_pdf(z, 0., 0.)
        logqz_x = log_normal_pdf(z, mean, logvar)
        total_loss -= tf.reduce_mean(logpx_z + logpz - logqz_x)
    return total_loss

In [ ]:
# @tf.function
def train_step(model, x, optimizer):
    z_prev = model.reparametrize(tf.zeros(shape = (1, 300)), tf.ones(shape = (1, 300)))
    x_prev = model.decode(z_prev)
    net_loss = 0
    # i = 0
    for train_samp in x:
        # if i == 0:
        #     continue
        train_sample = tf.reshape(train_samp, shape = (1, 1, 750))
        with tf.GradientTape() as tape:
            mean, logvar = model.encode(train_sample)
            z = model.reparametrize(mean, logvar)
            x_logit = model.decode(z)
            cross_ent = tf.nn.sigmoid_cross_entropy_with_logits(logits = x_logit, labels = train_sample)
            logpx_z = -tf.reduce_sum(cross_ent, axis = [1,2])
            logpz = log_normal_pdf(z, 0., 0.)
            logqz_x = log_normal_pdf(z, mean, logvar)
            loss_KL = -tf.reduce_mean(logpx_z + logpz - logqz_x)
            reconstruction_loss = tf.reduce_mean(tf.keras.losses.binary_crossentropy(train_sample, x_logit))
            mean_prev, logvar_prev = model.encode(x_prev)
            z_prev = model.reparametrize(mean_prev, logvar_prev)
            mean_pred, logvar_pred = model.get_next_state(z_prev)
            z_pred = model.reparametrize(mean_pred, logvar_pred)
            z_recon_loss = tf.reduce_mean(tf.keras.losses.binary_crossentropy(z, z_pred))
            total_loss = reconstruction_loss + loss_KL + z_recon_loss
        # print("Epoch " + str(i + 1)  + " : " + "Loss = " + str(total_loss))
        gradients = tape.gradient(total_loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))
        # i += 1
        # print("Epoch " + str(i + 1) + " done")
        x_prev = train_sample
        # print(total_loss)
        net_loss += total_loss
    return net_loss

In [ ]:
# train_dataset = (
#     tf.data.Dataset
#     .from_tensor_slices((TrackSet_1))
#     .map(map_data, num_parallel_calls=AUTOTUNE)
#     .shuffle(3)
#     .batch(BATCH_SIZE)
# )
# test_dataset = (
#     tf.data.Dataset
#     .from_tensor_slices((TrackSet_2))
#     .map(map_data, num_parallel_calls=AUTOTUNE)
#     .shuffle(3)
#     .batch(BATCH_SIZE)
# )

train_dataset = []

for filename in TrackSet_1:
    train_dataset.append(load(filename))

train_dataset = tf.convert_to_tensor(train_dataset, dtype = tf.float32)

test_dataset = []

for filename in TrackSet_2:
    test_dataset.append(load(filename))

test_dataset = tf.convert_to_tensor(test_dataset, dtype = tf.float32)

In [ ]:
train_dataset

<tf.Tensor: shape=(13, 100, 750), dtype=float32, numpy=
array([[[-0.00617013, -0.01385473, -0.01608861, ..., -0.02188793,
         -0.00172184, -0.00077854],
        [-0.00854354, -0.00394042, -0.0097504 , ...,  0.00419249,
         -0.00260758, -0.0056454 ],
        [ 0.00179191, -0.0011613 ,  0.00331267, ...,  0.01158473,
          0.01928655,  0.01505182],
        ...,
        [-0.00381679, -0.00500411, -0.01123971, ..., -0.0058227 ,
         -0.01284273, -0.02078624],
        [-0.01516015, -0.00926865, -0.00189545, ..., -0.00598077,
         -0.00934195, -0.01237829],
        [-0.01336239, -0.01239097, -0.0142961 , ..., -0.02049899,
         -0.01525505,  0.00053779]],

       [[ 0.01675216,  0.07342426, -0.12623996, ..., -0.13352165,
         -0.11601178, -0.01350737],
        [-0.05094993, -0.11805218, -0.06304592, ..., -0.1811176 ,
         -0.10112785,  0.01415266],
        [-0.00813589,  0.02537277,  0.07389332, ..., -0.22472794,
         -0.23269588, -0.28209594],
        ...

In [ ]:
test_dataset

<tf.Tensor: shape=(12, 100, 750), dtype=float32, numpy=
array([[[ 0.09052681,  0.26096714,  0.03838222, ...,  0.09505847,
         -0.22011402, -0.22248815],
        [ 0.18683311,  0.24498484, -0.01537771, ...,  0.13771997,
         -0.15418467, -0.07472448],
        [ 0.1431193 , -0.08900037, -0.31396025, ..., -0.17313775,
          0.00424118,  0.11987076],
        ...,
        [ 0.0323396 , -0.01239425, -0.06534199, ..., -0.06615368,
         -0.0652673 , -0.01371949],
        [ 0.05025498, -0.07314209, -0.03657861, ..., -0.01311526,
         -0.02016663,  0.00291737],
        [-0.01262713, -0.06105466, -0.10576949, ..., -0.16231762,
          0.02938618,  0.07584637]],

       [[-0.05478245, -0.11190942, -0.08331225, ..., -0.05193736,
          0.08008962,  0.13153963],
        [ 0.10029635,  0.02166242, -0.09242068, ..., -0.04909403,
         -0.07831205, -0.10820825],
        [-0.08485813, -0.03404973, -0.03481009, ...,  0.03372068,
         -0.12679571, -0.00947391],
        ...

In [50]:
def train(train_dataset, test_dataset, model):
    for epoch in range(1):
        start_time = time.time()
        for train_x in train_dataset:
            # train_x = np.asarray(train_x)[0][0]
            print(train_step(model, train_x, optimizer))
        end_time = time.time()

        # loss = tf.keras.metrics.Mean()
        # for test_x in test_dataset:
        #     test_x = np.asarray(test_x)[0][0]
        #     loss(compute_loss(model, test_x))
        # display.clear_output(wait=False)
        # elbo = -loss.result()
        # print('Epoch: {}, Test set ELBO: {}, time elapse for current epoch: {}'.format(epoch, 
        #                                                                                elbo, 
        #                                                                                end_time - start_time
        #                                                                                ))

In [51]:
model = VAE(300)
train(train_dataset, test_dataset, model)

tf.Tensor(51903.746, shape=(), dtype=float32)
tf.Tensor(50778.676, shape=(), dtype=float32)
tf.Tensor(48127.68, shape=(), dtype=float32)
tf.Tensor(42172.637, shape=(), dtype=float32)
tf.Tensor(32363.715, shape=(), dtype=float32)
tf.Tensor(21019.426, shape=(), dtype=float32)
tf.Tensor(11949.207, shape=(), dtype=float32)
tf.Tensor(6490.0054, shape=(), dtype=float32)
tf.Tensor(3418.0298, shape=(), dtype=float32)
tf.Tensor(1481.9987, shape=(), dtype=float32)
tf.Tensor(1466.9722, shape=(), dtype=float32)
tf.Tensor(-183.05017, shape=(), dtype=float32)
tf.Tensor(471.46555, shape=(), dtype=float32)


In [52]:
z_prev = model.reparametrize(tf.zeros(shape = (1, 300)), tf.ones(shape = (1, 300)))
new_music = []
for i in range(0, 100):
    mean, logvar = model.get_next_state(z_prev)
    z = model.reparametrize(mean, logvar)
    new_music.append(model.decode(z))
new_music = tf.convert_to_tensor(new_music)
new_music = tf.reshape(new_music, shape = (75000, )).numpy()
ipd.Audio(new_music, rate = 3000)

In [48]:
new_music

array([-14.58013 , -19.22847 , -25.748013, ..., -13.145725, -15.635441,
        -8.372599], dtype=float32)